# Fake News Detection
#### Akshay U
Its a Machine Learning program to find Fake news by training this system with Naive Bayes.

We have two datasets. `True.csv` and `Fake.csv`. <br>
True.csv contains only true news and Fake.csv contains only fake news.

### Import Libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score, f1_score

import joblib

### Location of Dataset
https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset

### Import data and Cleaning

In [2]:
true_df = pd.read_csv("True.csv")
fake_df = pd.read_csv("Fake.csv")

Check the dataset, its shape and basic info. 

Details of True.csv

In [3]:
true_df.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [4]:
true_df['title'][0]

'As U.S. budget fight looms, Republicans flip their fiscal script'

In [5]:
true_df['text'][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [6]:
true_df.shape

(21417, 4)

In [7]:
true_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
title      21417 non-null object
text       21417 non-null object
subject    21417 non-null object
date       21417 non-null object
dtypes: object(4)
memory usage: 669.4+ KB


Details of Fake.csv

In [8]:
fake_df.head(10)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017"
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017"
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017"
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017"


In [9]:
fake_df['title'][0]

' Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing'

In [10]:
fake_df['text'][0]

'Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev

In [11]:
fake_df.shape

(23481, 4)

In [12]:
fake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
title      23481 non-null object
text       23481 non-null object
subject    23481 non-null object
date       23481 non-null object
dtypes: object(4)
memory usage: 733.9+ KB


#### Check NaN
Check if any null oe NaN values in the dataset

In [13]:
true_df.isnull().values.any()

False

In [14]:
fake_df.isnull().values.any()

False

In [15]:
true_df.columns

Index(['title', 'text', 'subject', 'date'], dtype='object')

In [16]:
fake_df.columns

Index(['title', 'text', 'subject', 'date'], dtype='object')

We identified that this dataset is clean without NaN values. Also, we understand that we need only the `text` attribute for predicting the output. <br>
So we should divide the dataset columns for input and output.

Add a new column as `label` for store the news as REAL or FAKE.

Then concatinate the two dataframe to one for training.

In [17]:
true_df['label'] = "Real"
true_df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",Real
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",Real
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",Real
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",Real
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",Real


In [18]:
fake_df['label'] = "Fake"
fake_df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",Fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",Fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",Fake


In [19]:
df = pd.concat([true_df,fake_df])
df.shape

(44898, 5)

#### Inputs

In [20]:
X = df['text']
X

0        WASHINGTON (Reuters) - The head of a conservat...
1        WASHINGTON (Reuters) - Transgender people will...
2        WASHINGTON (Reuters) - The special counsel inv...
3        WASHINGTON (Reuters) - Trump campaign adviser ...
4        SEATTLE/WASHINGTON (Reuters) - President Donal...
                               ...                        
23476    21st Century Wire says As 21WIRE reported earl...
23477    21st Century Wire says It s a familiar theme. ...
23478    Patrick Henningsen  21st Century WireRemember ...
23479    21st Century Wire says Al Jazeera America will...
23480    21st Century Wire says As 21WIRE predicted in ...
Name: text, Length: 44898, dtype: object

#### Corresponding outputs

In [21]:
y = df['label']
y

0        Real
1        Real
2        Real
3        Real
4        Real
         ... 
23476    Fake
23477    Fake
23478    Fake
23479    Fake
23480    Fake
Name: label, Length: 44898, dtype: object

### Train Test Split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=99)
print("Training set - Features: ", X_train.shape, "  Target: ", y_train.shape)
print("Testing set  - Features: ", X_test.shape, "  Target: ",y_test.shape)

Training set - Features:  (35918,)   Target:  (35918,)
Testing set  - Features:  (8980,)   Target:  (8980,)


Lets check the split

In [23]:
X_train

15205    ABU DHABI/DUBAI (Reuters) - Saudi Arabia s cra...
7675     Today, Senator Bernie Sanders released a new c...
18701    ROME (Reuters) - The U.N. refugee agency is se...
12185    UNITED NATIONS (Reuters) - Japan s Foreign Min...
13118    BERLIN (Reuters) - German police on Tuesday ar...
                               ...                        
13119     How in the heck does the RNC keep getting the...
21280    It s time for Sweden to take off the blindfold...
14591    Thanks to Obama s open border policy and a rec...
2170     Donald Trump is starting another week in the W...
7896     The Republican party has been good at one thin...
Name: text, Length: 35918, dtype: object

In [24]:
y_train

15205    Real
7675     Fake
18701    Real
12185    Real
13118    Real
         ... 
13119    Fake
21280    Fake
14591    Fake
2170     Fake
7896     Fake
Name: label, Length: 35918, dtype: object

In [25]:
X_test

6170     It s not about polls, it s not about personali...
6474     TOKYO (Reuters) - U.S. President-elect Donald ...
13438    PARIS (Reuters) - French President Emmanuel Ma...
2216     Republicans nationwide seem to want to get rid...
18431    ERBIL, Iraq (Reuters) - Iraqi Kurdish leader J...
                               ...                        
4183     Tom Hanks is definitely voting for Hillary Cli...
15077    SEOUL (Reuters) - The leaders of South Korea a...
3202     One of the first things Donald Trump did after...
11831                                                     
20855    MUNICH, Germany (Reuters) - Victims of the att...
Name: text, Length: 8980, dtype: object

In [26]:
y_test

6170     Fake
6474     Real
13438    Real
2216     Fake
18431    Real
         ... 
4183     Fake
15077    Real
3202     Fake
11831    Fake
20855    Real
Name: label, Length: 8980, dtype: object

### Feature Extraction
<p style='text-align: justify'>Feature extraction is a process of dimensionality reduction by which an initial set of raw data is reduced to more manageable groups for processing. A characteristic of these large data sets is a large number of variables that require a lot of computing resources to process. Feature extraction is the name for methods that select and /or combine variables into features, effectively reducing the amount of data that must be processed, while still accurately and completely describing the original data set.</p>

Initialize a CountVectorizer with stop_words = 'english'. 

Then use **fit()** and store the result to a variable for make ***joblib*** file.

In [27]:
vect = CountVectorizer(stop_words='english')
vectorizer = vect.fit(X)

In [28]:
X_train_transformed = vect.transform(X_train)
X_test_transformed = vect.transform(X_test)
print("New Transformed...")
print("Training set - Features: ", X_train_transformed.shape, "  Target: ", y_train.shape)
print("Testing set  - Features: ", X_test_transformed.shape, "  Target: ",y_test.shape)

New Transformed...
Training set - Features:  (35918, 121690)   Target:  (35918,)
Testing set  - Features:  (8980, 121690)   Target:  (8980,)


### Modeling - Naive Bayes

In [29]:
def print_metrics(labels, preds):
    print("Precision Score\t: {}".format(precision_score(labels, preds, average='weighted')))
    print("Recall Score\t: {}".format(recall_score(labels, preds, average='weighted')))
    print("Accuracy Score\t: {}".format(accuracy_score(labels, preds)))
    print("F1 Score\t: {}".format(f1_score(labels, preds, average='weighted')))

In [30]:
mnb = MultinomialNB()
mnb.fit(X_train_transformed,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### Prediction and Accuracy

In [31]:
prediction = mnb.predict(X_test_transformed)
print_metrics(prediction, y_test)

Precision Score	: 0.9507852508651965
Recall Score	: 0.9507795100222717
Accuracy Score	: 0.9507795100222717
F1 Score	: 0.9507816243545234


### Train with full set of data -- 100% of data and store it for future prediction
We found the accuracy of this machine with 80% training data.<br>For the future prediction, we can train the machine with 100% dataset, which may increase the accuracy.

In [32]:
X_train_transformed = vect.transform(X)
naive = mnb.fit(X_train_transformed,y)

### Joblib File
Our dataset is little bit big, hence we are using ***joblib*** instead ***pickle***. Joblib file work similar to pickle file. And this file is using for future prediction and helps to avoid training the machine over again.

In [33]:
joblib.dump(naive,"naive.joblib")
joblib.dump(vectorizer,"vectoriszer.joblib")

['vectoriszer.joblib']